# Miller Building a Classifier
**Author:** Dan Miller

**Date:** November 2nd, 2025

**Objective:** Build and evaluate three classifiers using the Titanic dataset, then compare their performance across different feature sets in terms of predicting survival

## Introduction
This project explores the difference in performance between three classifiers: Decision Tree, Support Vector Machine, and Neural Network.  These classifiers will be made on the Titanic dataset to predict the feature 'survived'.  First, the data will be explored and there will be feature engineering done.  After that, each classifier will be made individually on three separate feature sets.  After all three classifiers are made and compared, there will be a summary at the end to discuss the findings.

## Section 1. Import and Inspect the Data

### 1.1 Import the necessary libraries

In [13]:
# Imports

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree

### 1.2 Load the dataset and display a few records

In [14]:
# Load Titanic dataset
titanic = sns.load_dataset("titanic")

titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Section 2. Data Exploration and Preparation

### 2.1 Handle Missing Values and Clean Data

The titanic dataset was already thoroughly explored in ml02_miller.ipynb, so we already know what needs to be done to the data.

In [15]:
# Impute missing values for age using the median

median_age = titanic["age"].median()
titanic["age"] = titanic["age"].fillna(median_age)

In [16]:
# Fill in missing embark_town values using the mode

mode_embark = titanic["embark_town"].mode()[0]
titanic["embark_town"] = titanic["embark_town"].fillna(mode_embark)

### 2.2 Feature Engineering

In [17]:
# Create new feature: family_size
titanic["family_size"] = titanic["sibsp"] + titanic["parch"] + 1

# Map categories to numeric values

titanic["sex"] = titanic["sex"].map({"male": 0, "female": 1})
titanic["embarked"] = titanic["embarked"].map({"C": 0, "Q": 1, "S": 2})

# Convert 'alone' to numeric binary
titanic["alone"] = titanic["alone"].astype(int)

## Section 3. Feature Selection and Justification

### 3.1 Choose features and target

While 'survived' will always be the target, there will be three different input cases:

Case 1:
- input feature: alone
- target: survived

Case 2:
- input features: age
- target: survived

Case 3:
- input features: age, family_size
- target: survived

### 3.2 Define X and y

In [18]:
# Case 1: Feature = alone

X1 = titanic[["alone"]]

y1 = titanic["survived"]

In [19]:
# Case 2: Feature = age (drop if na or not available)

X2 = titanic[["age"]].dropna()

y2 = titanic.loc[X2.index, "survived"]

In [20]:
# Case 3: Features = age & family_size (drop if na or not available)

X3 = titanic[["age", "family_size"]].dropna()

y3 = titanic.loc[X3.index, "survived"]